In [131]:
from solution_library import *

import numpy as np, matplotlib.pyplot as plt, pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import MinMaxScaler

import warnings

torch.set_default_dtype(torch.float64)

In [132]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(29, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [133]:
with warnings.catch_warnings(action="ignore"):
    x_df, y_df = claims_preprocess_get_xy()

In [134]:
x = x_df.values.astype(np.float64)
y = y_df.values.astype(np.float64)

scalar = MinMaxScaler() # USE THIS SAME SCALAR LATER
x_norm = scalar.fit_transform(x)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x_norm, y, test_size = 0.15, random_state=0)

inputs = torch.from_numpy(x_train)
targets = torch.from_numpy(y_train)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)

In [135]:
model = Net()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)
# optimizer = optim.SGD(model.parameters(), lr=0.025)


In [136]:
batch_size = 64
dataset = torch.utils.data.TensorDataset(inputs, targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

count = 0

for epoch in range(150):
    try:
        for inputs_batch, targets_batch in dataloader:
            optimizer.zero_grad()  # zero the parameter gradients
            outputs = model(inputs_batch)  # forward pass
            loss = torch.sqrt(criterion(outputs, targets_batch))  # compute loss
            loss.backward()  # backward pass
            optimizer.step()  # update weights

        # Evaluate on the validation set
        with torch.no_grad():
            val_outputs = model(x_test)
            val_loss = torch.sqrt(criterion(val_outputs, y_test))

        print(f'Epoch: {epoch + 1:4d}. Loss: {loss.item():.3f}. Validation Loss: {val_loss.item():.3f}')

        if loss.item() < 700 and count > 5:
            break
        elif loss.item() < 700:
            count += 1
            pass

    except KeyboardInterrupt:
        print('Ended Training Early')
        break
print('Finished Training')

c:\Users\kyle\OneDrive\Programs\Cooperators_2024\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2126])) that is different to the input size (torch.Size([2126, 1])). This will likely lead to i

Epoch:    1. Loss: 1282.689. Validation Loss: 1418.347
Epoch:    2. Loss: 900.066. Validation Loss: 1420.313
Epoch:    3. Loss: 1265.087. Validation Loss: 1420.427
Epoch:    4. Loss: 1416.563. Validation Loss: 1447.503
Epoch:    5. Loss: 589.413. Validation Loss: 1417.136
Epoch:    6. Loss: 697.050. Validation Loss: 1418.900
Epoch:    7. Loss: 1010.062. Validation Loss: 1419.074
Epoch:    8. Loss: 1005.585. Validation Loss: 1417.046
Epoch:    9. Loss: 1417.960. Validation Loss: 1421.723
Epoch:   10. Loss: 963.796. Validation Loss: 1420.586
Epoch:   11. Loss: 1256.523. Validation Loss: 1423.004
Epoch:   12. Loss: 866.458. Validation Loss: 1417.110
Epoch:   13. Loss: 771.442. Validation Loss: 1418.581
Epoch:   14. Loss: 1051.459. Validation Loss: 1416.945
Epoch:   15. Loss: 1019.094. Validation Loss: 1420.165
Epoch:   16. Loss: 771.279. Validation Loss: 1417.636
Epoch:   17. Loss: 1242.400. Validation Loss: 1417.257
Epoch:   18. Loss: 791.301. Validation Loss: 1417.265
Epoch:   19. Loss:

In [137]:
# for epoch in range(9000):
#     try:
#         outputs = model(inputs) # forward + backward + optimize        
#         loss = criterion(outputs, targets)

#         optimizer.zero_grad() # zero the parameter gradients
#         loss.backward()
#         optimizer.step()

#         print(f'Epoch: {epoch + 1:4d}. Loss: {loss:.3f}')

#         if loss < 1E-7:
#             break
#             # pass

#     except KeyboardInterrupt:
#         print('Ended Training Early')
#         break


# print('Finished Training')

In [138]:
x_new, ids = preprocess_x(load_tests())

x_new = x_new.values.astype(np.float64)

x_new_norm = scalar.transform(x_new)

inputs_pred = torch.from_numpy(x_new_norm)

model.eval() # convert to evaluation mode

with torch.no_grad(): # forward pass
    predictions = model(inputs_pred)

c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:119: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_pay_freq'] = claims['pol_pay_freq'].replace( {'Biannual': 2, 'Yearly': 1, 'Monthly': 12, 'Quarterly': 4} )
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:120: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_payd'] = claims['pol_payd'].replace( {'No': 0, 'Yes': 1} )
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:121: FutureWarning: Downcasting behav

In [139]:
rmse = RMSE(predictions, y_test)
rmse

tensor(1416.9009)

In [140]:
predictions = pd.DataFrame(predictions, columns=['claim_amount'])

save_data = pd.concat([ids, predictions], axis = 1)

save_data.to_csv(f'predictions/pred_nn_v1')

predictions

,claim_amount
0,1132.507558
1,1122.830905
2,1143.387248
3,1126.137521
4,1136.761373
...,...
4135,1131.069713
4136,1117.706367
4137,1134.945978
4138,1132.106309
